## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import pickle

In [2]:
# Assuming the file is named 'dataframe.pkl'
with open('dataframe.pkl', 'rb') as f:
    df = pickle.load(f)

## Código e discussão



### Divisão treino-teste

In [3]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
FEATURES = ['lcd', 'pld', 'void_fraction', 'surface_area_m2g', 'surface_area_m2cm3', 'CO2_0.01', 'CO2_0.1', 'CO2_2.5', 'CO2_0.05', 'CO2_0.5',
            'N2_0.09', 'N2_0.9', 'CH4_2.5', 'CH4_4.5', 'CH4_0.05', 'CH4_0.5', 'CH4_0.9', 'CH4_35', 'H2_2', 'H2_100']
TARGET = ['Xe_1']

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

In [4]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

In [5]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

In [8]:
y_prev = minha_MLP(X_treino)

In [9]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

In [10]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=20, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [11]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0437)
1 tensor(0.0405)
2 tensor(0.0374)
3 tensor(0.0347)
4 tensor(0.0325)
5 tensor(0.0306)
6 tensor(0.0289)
7 tensor(0.0274)
8 tensor(0.0260)
9 tensor(0.0248)
10 tensor(0.0238)
11 tensor(0.0230)
12 tensor(0.0223)
13 tensor(0.0217)
14 tensor(0.0212)
15 tensor(0.0208)
16 tensor(0.0204)
17 tensor(0.0199)
18 tensor(0.0195)
19 tensor(0.0190)
20 tensor(0.0184)
21 tensor(0.0178)
22 tensor(0.0171)
23 tensor(0.0164)
24 tensor(0.0157)
25 tensor(0.0150)
26 tensor(0.0144)
27 tensor(0.0139)
28 tensor(0.0133)
29 tensor(0.0128)
30 tensor(0.0123)
31 tensor(0.0118)
32 tensor(0.0112)
33 tensor(0.0106)
34 tensor(0.0100)
35 tensor(0.0094)
36 tensor(0.0088)
37 tensor(0.0083)
38 tensor(0.0077)
39 tensor(0.0072)
40 tensor(0.0068)
41 tensor(0.0063)
42 tensor(0.0059)
43 tensor(0.0055)
44 tensor(0.0052)
45 tensor(0.0048)
46 tensor(0.0045)
47 tensor(0.0042)
48 tensor(0.0039)
49 tensor(0.0036)
50 tensor(0.0033)
51 tensor(0.0031)
52 tensor(0.0028)
53 tensor(0.0026)
54 tensor(0.0025)
55 tensor(0.0023)
56

In [12]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.57337847] [0.56099122]
[1.553999] [1.58873526]
[0.72190402] [0.75441584]
[2.27605493] [2.29233926]
[3.03556895] [2.86371123]
[0.78030701] [0.82338169]
[2.368495] [2.38312213]
[2.65115801] [2.60807022]
[1.70751903] [1.71388069]
[0.95890204] [0.99854144]
[0.760066] [0.77471824]
[0.75630199] [0.75039491]
[4.25403996] [4.32266877]
[1.8873291] [1.83735467]
[2.61967902] [2.64236519]
[2.15343697] [2.1789104]
[0.63327298] [0.61319295]
[3.86579008] [3.87238605]
[2.45340706] [2.40721456]
[0.3991435] [0.40530478]
[1.692429] [1.69801076]
[0.78046498] [0.86513086]
[1.23384097] [1.13072327]
[0.937126] [0.95808683]
[1.71586797] [1.75856672]
[0.93059903] [0.85446894]
[1.02393604] [1.0348158]
[2.59136392] [2.59464311]
[0.65360399] [0.60473728]
[1.03309998] [1.03551861]
[0.931082] [0.92162502]
[3.46831008] [3.57596337]
[3.31250995] [3.25783883]
[0.60076829] [0.74421167]
[1.99398297] [2.03731025]
[4.56051013] [4.78277159]
[0.49076277] [0.47283732]
[0.64449702] [0.67761555]
[2.51258008] [2.57874101]
[3

In [13]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=20, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [14]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.74338086] [0.59822445]
[1.09562605] [1.13765396]
[0.00526056] [0.00554214]
[0.93107099] [0.96694012]
[0.96158997] [0.92117293]
[3.89166992] [3.90763767]
[3.0270899] [3.00797339]
[1.13198603] [1.1473732]
[0.91508726] [0.90007611]
[2.70047164e-13] [0.0093618]
[0.74763] [0.69732133]
[0.992004] [1.02825633]
[2.39291091] [2.63561719]
[1.82485507] [1.9319957]
[1.93217601] [2.04241897]
[0.0331167] [0.20582985]
[0.70727699] [0.68226305]
[1.27770099] [1.3300676]
[4.21562986] [4.53218012]
[0.77424353] [0.84971205]


In [15]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 0.11234068879160689
